In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
from collections import Counter
from random import randrange
from profanity_check import predict, predict_prob

In [2]:
songs = pd.read_csv('../data/songs_after_round_3_cleaning.csv')

# Build Lyrics Corpus

In [3]:
lyrics_corpus = ""

for l in list(songs['cleaned_lyrics']):
    lyrics_corpus += l

In [4]:
lyrics_list = lyrics_corpus.split()

In [5]:
lyrics_list = [s.replace("[", "") for s in lyrics_list]
lyrics_list = [s.replace("]", "") for s in lyrics_list]

# Get Probabilty Word Follows Another Word to Generate Lyrics

In [6]:
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(10)

In [7]:
dirty = ['fuck','ass','shit','bitch','cunt','nigga','nigger','niggas','niggers','asshole','penis','bitches']

In [8]:
def create_lyrics(start_word, length):
    
    new_word = start_word
    lyrics = start_word + ' '
    
    for i in range(length):
        
        following_words = []
        
        for i in range(len(lyrics_list)):
            if (lyrics_list[i] == new_word) and (i != len(lyrics_list) -1):
                following_words.append(lyrics_list[i+1])
    
        if len(set(following_words)) > 40:
            fol = most_frequent(following_words)[randrange(8)][0]
            if fol in dirty:
                fol = most_frequent(following_words)[randrange(7)][0]
            if fol in dirty:
                fol = most_frequent(following_words)[randrange(7)][0]
            lyrics += fol + " "
            new_word = fol
            
        else:
                lyrics += str(most_frequent(following_words)[0][0]) + " "
                new_word = most_frequent(following_words)[0][0]
                
       
            
        
    return(lyrics)

In [9]:
create_lyrics('friends', 12)

'friends lets get your heart i aint never been a couple hoes and '

In [10]:
create_lyrics('a', 12)

'a hundred on the world is the time that i cant believe me '

In [11]:
create_lyrics('and', 12)

'and a little more like the night to the same place for it '

In [12]:
create_lyrics('he', 12)

'he dont know i just like a lot for my way to my '

# Write Song Function

In [13]:
# list of first words in songs
starter = ['i','im','you','dont','every',
           'and','oh','yeah','youre','baby','so','she',
           'he','they','this','well','we','hey','if','how','hold','wanting','just','medicine','yesterday',
            'night','morning','sun','friend',
            'first','time','anxiety','loving']

In [14]:
def write_song(row_length):
    l = []
    for j in range(row_length):
        l.append(create_lyrics(starter[randrange(len(starter))], randrange(5,15)))
    return l

In [15]:
for thing in write_song(15):
    print(thing)

morning raint and we aint even the night show i got it up i 
we could take it was there with a lot i know what it 
loving on you want me a hundred bands 
hey im a new age to my name you and the same time 
she gon have it i dont care for a little closer to do 
this is you and i aint 
i can you cant stop thinking about you got no i 
im the time and you got the way i was born to do 
sun rise and my head to be alright girl you know you and im not 
hold it on that youre gone i dont know i aint tryna make my 
and my own way too long so much time we are 
i cant keep the one i was there with 
they say it was some people dont wanna go to the top off 
first you know it up a lot for the time that we 
night i cant keep that i was just 


In [16]:
for thing in write_song(7):
    print(thing)

morning im so i can be with my 
wanting my head and you got no i cant 
how i aint gotta say you want to do to 
first you and we gon make it to be my head to you got me 
hold on my head you and 
time and she dont even gotta be the night show up on your love 
youre gonna have you want it was 


# Adding Pegasus Text Summarization

In [17]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
import io
import seaborn as sb
from transformers import pipeline
from transformers import AutoModelWithLMHead, AutoTokenizer

In [18]:
len(lyrics_corpus)

4335131

In [ ]:
lyrics_corpus[0:500]

In [ ]:
model_name = 'google/pegasus-xsum'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [ ]:
batch = tokenizer.prepare_seq2seq_batch(lyrics_corpus[0:500], truncation=True, padding='longest', return_tensors='pt').to(torch_device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text[0])

In [ ]:
def write_song(row):
    batch = tokenizer.prepare_seq2seq_batch(row['cleaned_lyrics'], truncation=True, padding='longest', return_tensors='pt').to(torch_device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return str(tgt_text[0])

In [ ]:
list(songs['cleaned_lyrics'])[0]

In [ ]:
songs['pegasus'] = songs.apply(write_song, axis = 1)

In [ ]:
#Making pegasus summary and appending into a list
short_summary = []
batch = tokenizer.prepare_seq2seq_batch(lyrics_corpus[0:30], truncation=True, padding='longest', return_tensors='pt').to(torch_device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
short_summary.append(str(tgt_text[0]))